In [83]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()
Pkg.develop("QuantumDots")
using QuantumDots, QuantumDots.BlockDiagonals, LinearAlgebra
using Plots
#using Symbolics
using Latexify
#using LinearSolve # Solving for stationary state
#using OrdinaryDiffEq # For time evolution
using SymPy # For analytic solutions

  Activating project at `c:\Users\vi2823sv\.julia\dev\QuantumDots\examples`


**Table of contents**<a id='toc0_'></a>    
- [Intro to QuantumDots.jl](#toc1_)    
    - [Defining a basis](#toc1_1_1_)    
    - [Kitaev chain hamiltonian](#toc1_1_2_)    
    - [Majorana polarization](#toc1_1_3_)    
    - [Using SymPy for analytical solutions](#toc1_1_4_)    
    - [Transport](#toc1_1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Intro to QuantumDots.jl](#toc0_)

### <a id='toc1_1_1_'></a>[Defining a basis](#toc0_)
To specify a basis, define the indices and optionally a conserved quantum number


In [2]:
N=2
d = FermionBasis(1:N, (:↑, :↓); qn = QuantumDots.fermionnumber)

FermionBasis{4,Tuple{Int64, Symbol},SparseArrays.SparseMatrixCSC{Int64, Int64},QuantumDots.AbelianFockSymmetry{Vector{Int64}, Dictionaries.Dictionary{Int64, Int64}, Int64, typeof(QuantumDots.fermionnumber)}}:
keys = {(1, :↑), (2, :↑), (1, :↓), (2, :↓)}

The basis allows easy access to the fermionic operators in the many body basis, as well as dictionaries that define how the basis is ordered

In [3]:
d[1,:↑] #is a SparseArray representation of the annihilation operator

16×16 SparseArrays.SparseMatrixCSC{Int64, Int64} with 8 stored entries:
⎡⠈⠀⠠⡀⠀⠀⠀⠀⎤
⎢⠀⠀⠀⠀⠁⢀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠢⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠠⎦

In [4]:
Matrix(d[N,:↓]') # Creation operator. ' is hermitian conjugate. Use Matrix to convert to dense matrix.

16×16 Matrix{Int64}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0

In [5]:
d[1,:↑]'d[1,:↑] # Number operator 

16×16 SparseArrays.SparseMatrixCSC{Int64, Int64} with 8 stored entries:
⎡⠐⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠐⠄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠁⢀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠑⢀⎦

In [6]:
blockdiagonal(sum(f -> f'f, d), d) |> blocks .|> display; # Blocks of number operator

1×1 SparseArrays.SparseMatrixCSC{Int64, Int64} with 0 stored entries:
 ⋅

4×4 SparseArrays.SparseMatrixCSC{Int64, Int64} with 4 stored entries:
 1  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅
 ⋅  ⋅  1  ⋅
 ⋅  ⋅  ⋅  1

6×6 SparseArrays.SparseMatrixCSC{Int64, Int64} with 6 stored entries:
 2  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  2  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  2  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  2  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  2  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  2

4×4 SparseArrays.SparseMatrixCSC{Int64, Int64} with 4 stored entries:
 3  ⋅  ⋅  ⋅
 ⋅  3  ⋅  ⋅
 ⋅  ⋅  3  ⋅
 ⋅  ⋅  ⋅  3

1×1 SparseArrays.SparseMatrixCSC{Int64, Int64} with 1 stored entry:
 4

5-element Vector{Nothing}:
 nothing
 nothing
 nothing
 nothing
 nothing

In [7]:
d.symmetry.qntofockstates

5-element Dictionaries.Dictionary{Int64, Vector{Int64}}
 0 │ [0]
 1 │ [1, 2, 4, 8]
 2 │ [3, 5, 6, 9, 10, 12]
 3 │ [7, 11, 13, 14]
 4 │ [15]

In [8]:
d.symmetry.indtofockdict

16-element Vector{Int64}:
  0
  1
  2
  4
  8
  3
  5
  6
  9
 10
 12
  7
 11
 13
 14
 15

In [9]:
print(rand(4^N), d)

labels = |(1, :↑)(2, :↑)(1, :↓)(2, :↓)>
QN = 0
 |0000> : 0.956
QN = 1
 |1000> : 0.739
 |010

0> : 0.927
 |0010> : 0.037
 |0001> : 0.693
QN = 2
 |1100> : 0.313
 |1010> : 0.298
 |0110> : 0.176
 |1001> : 0.043
 |0101> : 0.681
 |0011> : 0.865
QN = 3
 |1110> : 0.692
 |1101> : 0.967
 |1011> : 0.777
 |0111> : 0.133
QN = 4
 |1111> : 0.291


Any quantum number which is diagonal in the fock basis is supported

In [10]:
uniqn(focknbr) =
    if iseven(focknbr)
        return :🦄
    else
        if focknbr > 2
            return :👹
        else
            return :🌯
        end
    end
d2 = FermionBasis(1:2; qn=uniqn)

FermionBasis{2,Int64,SparseArrays.SparseMatrixCSC{Int64, Int64},QuantumDots.AbelianFockSymmetry{Vector{Int64}, Dictionaries.Dictionary{Int64, Int64}, Symbol, typeof(uniqn)}}:
keys = {1, 2}

In [11]:
print(rand(2^length(d2)), d2)

labels = |12>
QN = 

🌯
 |10> : 0.435
QN = 👹
 |11> : 0.29
QN = 🦄
 |00> : 0.903
 |01> : 0.9


There is also a free fermion version

In [12]:
b = QuantumDots.FermionBdGBasis(1:N, (:↑, :↓))
sum(f -> f'f, b)

8×8 SparseArrays.SparseMatrixCSC{Int64, Int64} with 8 stored entries:
 1  ⋅  ⋅  ⋅   ⋅   ⋅   ⋅   ⋅
 ⋅  1  ⋅  ⋅   ⋅   ⋅   ⋅   ⋅
 ⋅  ⋅  1  ⋅   ⋅   ⋅   ⋅   ⋅
 ⋅  ⋅  ⋅  1   ⋅   ⋅   ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅  -1   ⋅   ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅   ⋅  -1   ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅   ⋅   ⋅  -1   ⋅
 ⋅  ⋅  ⋅  ⋅   ⋅   ⋅   ⋅  -1

### <a id='toc1_1_2_'></a>[Kitaev chain hamiltonian](#toc0_)
We'll implement the interacting Kitaev chain hamiltonian
$$
\begin{equation}
H = \sum_{n = 1}^N \mu c_n^\dagger c_n + \sum_{n=1}^{N-1} (t c_{n+1}^\dagger c_n + \Delta c_{n+1}c_n + \mathrm{h.c}) + V c_{n+1}^\dagger c_{n+1} c_n^\dagger c_n,
\end{equation}
$$
and calculate the energy gap and majorana polarization. We'll use these to tune for a sweet spot where the system has two separated majoranas.

In [72]:
N = 2
c = FermionBasis(1:N, qn=QuantumDots.parity)
cBdG = QuantumDots.FermionBdGBasis(1:N)
ham(μ, t, Δ, V, c=c) = Hermitian(sum(μ * c[n]'c[n] for n in 1:N) + sum(t * c[n+1]'c[n] + Δ * c[n+1]c[n] + hc for n in 1:N-1) .+ ham_int(V, c))
ham_int(V, c) = V * sum(c[n]'c[n]c[n+1]'c[n+1] for n in 1:N-1)
ham_int(V, c::QuantumDots.FermionBdGBasis) = 0

ham_int (generic function with 2 methods)

In [73]:
@syms μ::real t::real Δ::real V::real

(μ, t, Δ, V)

In [82]:
#Many body hamiltonian
ham(μ, t, Δ, V) # |> latexify |> println # Uncomment to print latex code 

4×4 Hermitian{Sym, SparseArrays.SparseMatrixCSC{Sym, Int64}}:
 μ  t   0        0
 t  μ   0        0
 0  0   0       -Δ
 0  0  -Δ  V + 2*μ

In [75]:
ham(μ, t, Δ, V, cBdG) #BdG hamiltonian

4×4 Hermitian{Sym, SparseArrays.SparseMatrixCSC{Sym, Int64}}:
 μ   t   0   Δ
 t   μ  -Δ   0
 0  -Δ  -μ  -t
 Δ   0  -t  -μ

In [76]:
ham(0.5, 1.0, 0.5, 0.2)

4×4 Hermitian{Float64, SparseArrays.SparseMatrixCSC{Float64, Int64}}:
 0.5  1.0   0.0   0.0
 1.0  0.5   0.0   0.0
 0.0  0.0   0.0  -0.5
 0.0  0.0  -0.5   1.2

#### Diagonalizing the Hamiltonian

#### <a id='toc1_1_3_'></a>[Majorana quality measures](#toc0_)
The Majorana Polarization $MP_R$ is a measure of how much the left and right majorana overlap in the region R. The larger it is, the smaller the overlap between the majoranas.
$$\begin{align*}
w_n &= \lang \psi_o | (c_n^\dagger + c_n)  | \psi_e \rang \\
z_n &= \lang \psi_o | (c_n^\dagger - c_n)  | \psi_e \rang \\
MP_R &= \left| \sum_{n \in R} w_n^2 - z_n^2 \right|
\end{align*}$$


In [28]:
function MP(ψₒ, ψₑ, R, c=c)
    w = map(f -> ψₒ' * (f' + f) * ψₑ, c)
    z = map(f -> ψₒ' * (f' - f) * ψₑ, c)
    abs(sum(w[n]^2 - z[n]^2 for n in R))
end

MP (generic function with 2 methods)

The Local Distinguishability $LD_R$ measures how much the even and odd ground states can be distinguished based on measurements in the region R. 
$$\begin{align*}
\delta \rho &= |\psi_o \rang \lang \psi_o | - |\psi_e \rang \lang \psi_e | \\
LD_R &=  \| \operatorname{Tr}_{R^c}[\delta\rho] \|
\end{align*}$$

In [6]:
function LD(ψₒ, ψₑ, R, c=c)
    δρ = ψₒψₒ' - ψₑψₑ'
    norm(reduced_density_matrix(δρ, R, c))
end

LD (generic function with 2 methods)

In [111]:
MP(vo,ve,c,[1])

|                                             2                               
|/                  2*Δ                      \    /                  2*Δ      
||--------------------------------------- - 1|  - |---------------------------
||             __________________________    |    |             ______________
||            /  2              2      2     |    |            /  2           
|\V + 2*μ - \/  V  + 4*V*μ + 4*Δ  + 4*μ      /    \V + 2*μ - \/  V  + 4*V*μ + 

                 2|
                \ |
------------ + 1| |
____________    | |
   2      2     | |
4*Δ  + 4*μ      / |

### <a id='toc1_1_4_'></a>[Using SymPy for analytical solutions](#toc0_)

In [91]:
using SymPy
@syms μ::real t::real Δ::real V::real
H = blockdiagonal(ham(μ, t, Δ, V), c)


4×4 BlockDiagonal{Sym, Hermitian{Sym, SparseArrays.SparseMatrixCSC{Sym, Int64}}}:
 μ  t   0        0
 t  μ   0        0
 0  0   0       -Δ
 0  0  -Δ  V + 2*μ

In [92]:
Mo, Me = sympy.Matrix.(Matrix.(blocks(H)))

2-element Vector{Matrix{Sym}}:
 [μ t; t μ]
 [0 -Δ; -Δ V + 2*μ]

In [93]:
vecs_o, vals_o = Mo.diagonalize()
vecs_e, vals_e = Me.diagonalize()

(Sym[-2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) -2*Δ/(V + 2*μ + sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)); 1 1], Sym[V/2 + μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)/2 0; 0 V/2 + μ + sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)/2])

In [63]:
Ee = diag(vals_e)[1]
ve = vcat(zero(vecs_e[:, 1]), vecs_e[:, 1])

4-element Vector{Sym}:
                                                  0
                                                  0
 -2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2))
                                                  1

In [65]:
Eo = diag(vals_o)[1]
vo = vcat(vecs_o[:, 1], zero(vecs_o[:, 1]))

4-element Vector{Sym}:
 -1
  1
  0
  0

In [94]:
mp = majorana_polarization(majorana_coefficients(vo, ve, c)..., (1,))

(mp = Abs(-(-2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)^2 + (2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)^2)/((-2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)*(-2*Δ/(V + 2*μ - conjugate(sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2))) - 1) + (2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)*(2*Δ/(V + 2*μ - conjugate(sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2))) - 1)), mpu = Abs(-(-2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)^2 + (2*Δ/(V + 2*μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)) - 1)^2))

In [98]:
latexify(mp.mpu)

L"$\mathrm{Abs}\left(  - \left( \frac{-2 \cdot \Delta}{V + 2 \cdot \mu - \sqrt{V^{2} + 4 \cdot V \cdot \mu + 4 \cdot \Delta^{2} + 4 \cdot \mu^{2}}} - 1 \right)^{2} + \left( \frac{2 \cdot \Delta}{V + 2 \cdot \mu - \sqrt{V^{2} + 4 \cdot V \cdot \mu + 4 \cdot \Delta^{2} + 4 \cdot \mu^{2}}} - 1 \right)^{2} \right)$"

In [97]:
LD = norm(reduced_density_matrix(vo * vo' - ve * ve', (1,), c))

|                                           2                                 
|                                        4*Δ                                  
|-----------------------------------------------------------------------------
|                                          /          ________________________
|/             __________________________\ |             _____________________
||            /  2              2      2 | |            /  2              2   
|\V + 2*μ - \/  V  + 4*V*μ + 4*Δ  + 4*μ  /*\V + 2*μ - \/  V  + 4*V*μ + 4*Δ  + 

          |
          |
------ - 1|
_____\    |
_____|    |
   2 |    |
4*μ  /    |

In [126]:
diagH = QuantumDots.DiagonalizedHamiltonian(vcat(diag(vals_o),diag(vals_e)),BlockDiagonal([Mo,Me]))

QuantumDots.DiagonalizedHamiltonian{Vector{Sym}, BlockDiagonal{Sym, Matrix{Sym}}}(Sym[-t + μ, t + μ, V/2 + μ - sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)/2, V/2 + μ + sqrt(V^2 + 4*V*μ + 4*Δ^2 + 4*μ^2)/2], Sym[μ t 0 0; t μ 0 0; 0 0 0 -Δ; 0 0 -Δ V + 2*μ])

In [29]:
eigblocks = blocks(diagH; full=true)
ground_states = map(QuantumDots.ground_state, eigblocks)
ϕo = ground_states[1].vector
ϕe = ground_states[2].vector

16-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.683539171799759
  0.3003347769685026
 -0.2900394004762402
  0.3198021876132616
  0.2557644093310968
 -0.2900394004762404
  0.3003347769685026
  0.1282094542610353

In [52]:
δρ = ϕe*ϕe' - ϕo*ϕo'
LD = norm(reduced_density_matrix(δρ,(1,),c))

0.05936079886511918

In [111]:
coeffs = majorana_coefficients(ground_states...,c)
mpu = majorana_polarization(coeffs..., (1,)).mpu

0.7985341677281106

In [53]:
function solve(H)
    diagH = QuantumDots.diagonalize(H)
    eigblocks = blocks(diagH; full=true)
    ((Eo,ϕo), (Ee,ϕe)) = map(QuantumDots.ground_state, eigblocks)
    ρe = ϕe*ϕe'
    ρo = ϕo*ϕo'
    δρ = ϕo-ϕe
    gap = Eo-Ee
    w,z = majorana_coefficients(ϕo, ϕe, c)
    R = Tuple(1:div(N,2))
    mpu = majorana_polarization(w,z, R).mpu
    LD = reduced_density_matrix(δρ, R, c)
    return gap, mpu, LD
end
solve(μ,t,Δ,V) = solve(kitaev_hamiltonian(μ,t,Δ,V))

solve (generic function with 2 methods)

In [54]:
solve(1.0,1,2,1.0)

(0.49597691398647026, 0.7324678436633643, [1.048092283813159 0.2603920647636982 -0.20366826500294938 0.5302545115518197; 0.2603920647636982 0.28740508272532744 -0.32773471719152725 0.1112530439384502; -0.20366826500294938 -0.32773471719152725 0.3929888228890849 -0.07508449798197797; 0.5302545115518197 0.1112530439384502 -0.07508449798197797 0.2715138105724287])

In [55]:
function cost(μ,t) 
    gap, mpu, LD = solve(μ,t,1.0,.5)
    10^3*gap^2 + 1-mpu
end
cost(args) = cost(args...)

cost (generic function with 2 methods)

In [56]:
using BlackBoxOptim

In [57]:
opt = bboptimize(cost; NumDimensions = 2)

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64}, RadiusLimitedSelector, BlackBoxOptim.AdaptiveDiffEvoRandBin{3}, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps


0.50 secs, 3999 evals, 3886 steps, improv/step: 0.161 (last = 0.1611), fitness=0.000000004


1.00 secs, 8202 evals, 8091 steps, improv/step: 0.161 (last = 0.1605), fitness=0.000000004



Optimization stopped after 10001 steps and 1.23 seconds
Termination reason: Max number of steps (10000) reached
Steps per second = 8097.98
Function evals per second = 8187.85
Improvements/step = 0.13760
Total function evaluations = 10112


Best candidate found: [-0.205806, -0.754626]

Fitness: 0.000000004



BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (10000) reached", 10001, 1.694598612864e9, 1.2349998950958252, ParamsDictChain[ParamsDictChain[Dict{Symbol, Any}(:RngSeed => 688271, :NumDimensions => 2, :MaxSteps => 10000),Dict{Symbol, Any}()],Dict{Symbol, Any}(:CallbackInterval => -1.0, :TargetFitness => nothing, :TraceMode => :compact, :FitnessScheme => ScalarFitnessScheme{true}(), :MinDeltaFitnessTolerance => 1.0e-50, :NumDimensions => :NotSpecified, :FitnessTolerance => 1.0e-8, :TraceInterval => 0.5, :MaxStepsWithoutProgress => 10000, :MaxSteps => 10000…)], 10112, ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64, Vector{Float64}}(4.032772005224672e-9, [-0.20580625919641177, -0.754626089690799]), BlackBoxOptim.PopulationOptimizerOutput{FitPopulation{Float64}}(FitPopulation{Float64}([-0.2058064215810058 -0.20580561355895347 … -0.2058072355475354 -0.20580661252628726; -0.7546260754141363 -0.754626209129304 … -0.75462

In [102]:
best_candidate(opt)

2-element Vector{Float64}:
  0.43289240307001087
 -0.26754221874193773

### <a id='toc1_1_5_'></a>[Transport](#toc0_)

In [124]:
function get_leads(c, T, μ, Γ=1)
    N = QuantumDots.nbr_of_fermions(c)
    left = QuantumDots.NormalLead(Γ * c[1]'; T, μ=μ[1])
    right = QuantumDots.NormalLead(Γ * c[N]'; T, μ=μ[2])
    return (; left, right)
end
PauliSystem(H,leads) = QuantumDots.Pauli()(QuantumDots.OpenSystem(H, leads))

PauliSystem (generic function with 1 method)

In [134]:
@syms V1::real V2::real T::real
symleads = get_leads(c,T,(V1,V2));

In [140]:
paulisystem = QuantumDots.Pauli()(QuantumDots.diagonalize(QuantumDots.OpenSystem(diagH, symleads)));

In [143]:
lindbladsystem = QuantumDots.Lindblad()(QuantumDots.diagonalize(QuantumDots.OpenSystem(diagH, symleads)));

In [147]:
Matrix(lindbladsystem);

In [154]:
function conductance(H,T,V1,V2)
    # H = kitaev_hamiltonian(μ,t,Δ,V,T)
    leads = get_leads(c,T,(V1,V2))
    sys = QuantumDots.Pauli()(QuantumDots.diagonalize(QuantumDots.OpenSystem(H, leads)));
    conductance_matrix(sys)
end

conductance (generic function with 1 method)

In [155]:
conductance(ham(1.0,1,1,1), 1,0,0)

MethodError: MethodError: no method matching init(::SciMLBase.LinearProblem{Vector{Float64}, true, SciMLOperators.MatrixOperator{Float64, Matrix{Float64}, SciMLOperators.FilterKwargs{typeof(SciMLOperators.DEFAULT_UPDATE_FUNC), Tuple{}}, SciMLOperators.FilterKwargs{typeof(SciMLOperators.DEFAULT_UPDATE_FUNC), Tuple{}}}, Vector{Float64}, SciMLBase.NullParameters, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}})

Closest candidates are:
  init(!Matched::SciMLBase.OptimizationProblem, !Matched::Any, !Matched::Any...; kwargs...)
   @ SciMLBase C:\Users\svens\.julia\packages\SciMLBase\l4PVV\src\solve.jl:146


In [136]:
conductance_matrix(system)

MethodError: MethodError: no method matching init(::SciMLBase.LinearProblem{Vector{Sym}, true, SciMLOperators.MatrixOperator{Sym, Matrix{Sym}, SciMLOperators.FilterKwargs{typeof(SciMLOperators.DEFAULT_UPDATE_FUNC), Tuple{}}, SciMLOperators.FilterKwargs{typeof(SciMLOperators.DEFAULT_UPDATE_FUNC), Tuple{}}}, Vector{Sym}, SciMLBase.NullParameters, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}})

Closest candidates are:
  init(!Matched::SciMLBase.OptimizationProblem, !Matched::Any, !Matched::Any...; kwargs...)
   @ SciMLBase C:\Users\svens\.julia\packages\SciMLBase\l4PVV\src\solve.jl:146


In [125]:
PauliSystem(H,leads)

MethodError: MethodError: no method matching eigen!(::Hermitian{Sym, Matrix{Sym}})

Closest candidates are:
  eigen!(::Union{Hermitian{T, var"#s969"}, Hermitian{Complex{T}, var"#s969"}, Symmetric{T, var"#s969"}} where var"#s969"<:(StridedMatrix{T} where T), !Matched::AbstractMatrix{T}; sortby) where T<:Number
   @ LinearAlgebra C:\Users\svens\.julia\juliaup\julia-1.9.3+0.x64.w64.mingw32\share\julia\stdlib\v1.9\LinearAlgebra\src\symmetriceigen.jl:167
  eigen!(!Matched::StridedMatrix{T}; permute, scale, sortby) where T<:Union{Float32, Float64}
   @ LinearAlgebra C:\Users\svens\.julia\juliaup\julia-1.9.3+0.x64.w64.mingw32\share\julia\stdlib\v1.9\LinearAlgebra\src\eigen.jl:149
  eigen!(!Matched::StridedMatrix{T}; permute, scale, sortby) where T<:Union{ComplexF64, ComplexF32}
   @ LinearAlgebra C:\Users\svens\.julia\juliaup\julia-1.9.3+0.x64.w64.mingw32\share\julia\stdlib\v1.9\LinearAlgebra\src\eigen.jl:172
  ...
